In [1]:
import pandas as pd
import numpy as np

In [41]:
# Making arrays, input(x) and output(y) for or gate
# x1 x2 y
# 0  0  0
# 0  1  1
# 1  0  1
# 1  1  1

# Therefore x array is 2d array(4*2) and y array is 1d array(4,1)

x = np.array([[0,0],[0,1],[1,0],[1,1]])
print(x.shape)

y = np.array([0,1,1,1])
print(y.shape)

(4, 2)
(4,)


In [185]:
# calcuting entropy for a particular node with total nodes , class A nodes and class B nodes passed as arguments
# import math for calculating log

import math

def entropy(totalSamples,aSamples,bSamples):
    if aSamples == totalSamples: # class A pure node
        return 0.0
    elif bSamples == totalSamples: # class B pure node
        return 0.0
    else :
        pA = aSamples / totalSamples
        pB = bSamples / totalSamples
        log_pA = math.log(pA,2)
        log_pB = math.log(pB,2)
        
        # Entropy or info_Node
        info_Node = - (pA * log_pA) - (pB * log_pB)
        return info_Node
    

In [186]:
c = entropy(4,3,1)
print(c)

0.8112781244591328


In [119]:
 def current_entropy(x,y):
        totalSamples = len(y)
        classA = 0
        classB = 0
        for i in range(totalSamples):
            if y[i]==1:
                classA += 1
            else :
                classB += 1
            
        gain_Current_Node = entropy(totalSamples,classA,classB)
        return gain_Current_Node,classA,classB

 We will be using same function for entropy as well as split_info. Just arguments will be different.
 
 1.rightSamples : samples going to the right node 
 
 2.leftSamples : samples going to the left node

 3.So, if we pass rightSamples and leftSamples in entropy function instead of classes(aSample, bSample) sample, we will get split_info instead of entropy 

In [190]:
# defining gain ration function with three arguments , x (input), y(output), feat ind : feature (index) under consideration, entropy of current node

def gain_ratio(x,y,feat_ind,gain_Current_Node):
    # calculate gain for current node before splitting
    totalSamples = len(y)
    
    rightSamples = 0 # count of samples going in right node 
    leftSamples = 0 # count of samples going in left node
    # the above variables needed to calculate split info
    
    cr1 = 0 # count of 1s in right node where x[feat_ind][] will be false
    cr0 = 0 # count of 0s i right node where x[feat_ind][] will be false
    # the above two will be needed to calculate entropy for right node
    
    cl1 = 0 # count of 1s in left node where x[feat_ind][] will be true
    cl0 = 0 # count of 0s in left node where x[feat_ind][] will be true
    # the above two will be needed to calculate entropy for left node
    
    for j in range(totalSamples):
        if x[j][feat_ind] == 1:
            leftSamples = leftSamples + 1;
            if y[j] == 1:
                cl1 = cl1 + 1
            else:
                cl0 = cl0 + 1
        else :
            rightSamples = rightSamples + 1
            if y[j] == 1:
                cr1 = cr1 + 1
            else :
                cr0 = cr0 + 1
    
    # Entropy of right subNode : entRight
    # Entropy of left subNode : entLeft
    entRight = entropy(rightSamples,cr1,cr0)
    entLeft = entropy(leftSamples,cl1,cl0)
    
    # Info Gain(entropy) of split nodes without considering split info [using weighted addition]
    gain_Split_Nodes = ((rightSamples/totalSamples) * entRight)+ ((leftSamples/totalSamples) * entLeft);
    
    # gain = original_node_entropy - split_nodes_entropy
    gain = gain_Current_Node - gain_Split_Nodes;
    
    # Calculating split info for consideration in gain_ratio formula
    split_info = entropy(totalSamples,rightSamples,leftSamples)
    
    if split_info != 0:
        gain_ratio = gain / split_info
    else :
        gain_ratio = gain
        
    return gain_ratio

In [191]:
# checking gain ratio function
# ans 1 for choosing x0 
# ans 1 is a pair consisting of (gain ratio and current entropy)
gain_Current = current_entropy(x,y);

ans1 = gain_ratio(x,y,0,gain_Current[0])
print(ans1)
ans2 = gain_ratio(x,y,1,gain_Current[0])
print(ans2)

# both are same for or gate

0.31127812445913283
0.31127812445913283


In [201]:
# Our recursive decision tree
# arguments passed are 
# 1) x for input
# 2) y for output
# 3) counterLevel stating level of decision tree, we have reached
# 4) features_added : telling which features have been added for printing right index 

def recursiveDT(x,y,counterLevel,features_added):
    total_features = x.shape[1]
    
    max_gain = -527534723572
    ans_feature = -1
    
    gain_Current = current_entropy(x,y)
    
    
    # Calculating which feature to split on
    for i in range(total_features):
        ans1 = gain_ratio(x,y,i,gain_Current[0])
        
        if ans1 > max_gain:
            max_gain = ans1
            ans_feature = i
            
    # if a feature found
    if ans_feature !=-1:
        print("Level ",counterLevel)
        counterLevel += 1
        
        if(gain_Current[1]!=0):
            print("Number of Ones(True) : ",gain_Current[1])
        if(gain_Current[2]!=0):
            print("Number of Zeros(False) : ",gain_Current[2])
        
        print("Current Entropy is : ",gain_Current[0])
        
        # if entropy not equal to zero
        if(gain_Current[0]!=0):
            if ans_feature in features_added:
                print("Splitting on feature X",ans_feature + 1," with gain ratio ")
            else:
                print("Splitting on feature X",ans_feature," with gain ratio ")
                features_added[ans_feature] = True
            print(max_gain)
        else: # incase entropy equal to zero , means leaf node
            print("Reached Leaf Node.")
        
        print()
        
        #First creating empty lists to breakdown our inputs according to the feature decided
        
        x1 = [[]]
        x2 = [[]]
        y1 =[]
        y2 = []
        
        for i in range(len(x)):
            var = np.ndarray.tolist(x[i]) # converting array to list
            del var[ans_feature] # removing the feature enteries,that we have already calculated
            if x[i][ans_feature]==1:
                x1.append(var)
                y1.append(y[i])
            else :
                x2.append(var)
                y2.append(y[i])
    
    # converting the lists to arrays
        x1.remove([])
        x2.remove([])
    
        x1 = np.array(x1)
        x2 = np.array(x2)
        y1 = np.array(y1)
        y2 = np.array(y2)
        
        if(gain_Current[0]!=0): # call recursively if current entropy not equal to zero
            recursiveDT(x2,y2,counterLevel,features_added)
            recursiveDT(x1,y1,counterLevel,features_added)
    
        return
    
    else: # if No feature to split upon
        
        print("Level ",counterLevel)
        counterLevel += 1
        
        if gain_Current[1]!=0:
            print("Number of Ones(True) : ",gain_Current[1])
        if gain_Current[2]!=0:
            print("Number of Zeros(False) : ",gain_Current[2])
        
        print("Current Entropy is : ",gain_Current[0])
        
        # if entropy not equal to zero
        if(gain_Current[0]!=0):
            if ans_feature in features_added:
                print("Splitting on feature X",ans_feature + 1," with gain ratio ")
            else:
                print("Splitting on feature X",ans_feature," with gain ratio ")
                features_added[ans_feature] = True
            print(max_gain)
            
        else: # if entropy is zero that means we are at a leaf node
            print("Reached Leaf Node.")
        print()
        return    

In [202]:
features_added = {} # to keep which features are added for recursive calls
# idea is that if a feature in a recursive call is already in features_added dictionary then the index of feature will be
# ans_feature = ans_feature + 1


recursiveDT(x,y,0,features_added)

Level  0
Number of Ones(True) :  3
Number of Zeros(False) :  1
Current Entropy is :  0.8112781244591328
Splitting on feature X 0  with gain ratio 
0.31127812445913283

Level  1
Number of Ones(True) :  1
Number of Zeros(False) :  1
Current Entropy is :  1.0
Splitting on feature X 1  with gain ratio 
1.0

Level  2
Number of Zeros(False) :  1
Current Entropy is :  0.0
Reached Leaf Node.

Level  2
Number of Ones(True) :  1
Current Entropy is :  0.0
Reached Leaf Node.

Level  1
Number of Ones(True) :  2
Current Entropy is :  0.0
Reached Leaf Node.

